In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [4]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import svm
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, f1_score

## Get Data and split into train & test sets

In [ ]:
# import pandas as pd
# path = "/content/drive/MyDrive/Data/"
# data = path + 'preprocessedData.csv'
# df = pd.read_csv(data, encoding='utf-8')

In [ ]:
# df.columns

In [ ]:
# train, test = train_test_split(df, test_size= 0.2, random_state = 42, stratify= df['label'])

In [ ]:
# import csv

# train.to_csv(path + 'train.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
# test.to_csv(path + 'test.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')

In [6]:
# get train set
path = "/content/drive/MyDrive/"
training_data = path + 'train.csv'
trDf = pd.read_csv(training_data, encoding='utf-8')

In [7]:
trDf = trDf.loc[trDf.label_name != 'Mixed' ]
trDf.label_name.value_counts()

Negative    5907
Neutral     5515
Positive    3520
Name: label_name, dtype: int64

In [8]:
trDf.columns

Index(['Tweet_text', 'label', 'label_name', 'clean_text_withEmojies',
       'clean_text', 'noStopWords', 'normalized_text', 'stemmed_farasa',
       'norm_stemm_farasa', 'stemmed_text_ISRI', 'norm_stemmISRI'],
      dtype='object')

In [9]:
# get test set
testing_data = path + 'test.csv'
tsDf = pd.read_csv(testing_data, encoding='utf-8')
tsDf = tsDf.loc[tsDf.label_name != 'Mixed' ]
tsDf.label_name.value_counts()

Negative    1477
Neutral     1379
Positive     880
Name: label_name, dtype: int64

In [10]:
y_train = trDf['label']
y_test = tsDf['label']

In [11]:
X_tr = trDf['norm_stemmISRI'].values.astype('U')
X_ts = tsDf['norm_stemmISRI'].values.astype('U')

### Vectorization

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

 ------ change it to do tf-idf vectorizing on just word level

In [ ]:
# count_vect = CountVectorizer()
# X_train_counts = count_vect.fit_transform(X_train)
# X_test_counts = count_vect.transform(X_test)

# tfidf_transformer = TfidfTransformer()
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [13]:
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer



vectorizer = TfidfVectorizer(ngram_range=(1, 2)) # ngram_range=(1, 2)

X_train = vectorizer.fit_transform(X_tr)
# X_train = X_train.toarray()
X_test = vectorizer.transform(X_ts)
# dense_tridf_vectors = X_test.toarray()
#  1
#x=trDf['Tweet_text']
#vectorizer = TfidfVectorizer(analyzer='word')
#vectorizer.fit(trDf)
#tridf_vectors = vectorizer.transform(trDf)

#dense_tridf_vectors = tridf_vectors.toarray()
#dense_tridf_vectors= dense_tridf_vectors.flatten()


In [ ]:
# X_train.shape
# print(dense_tsidf_vectors)

### training and testing the model

In [14]:
category_id_df = trDf[['label_name', 'label']].drop_duplicates().sort_values('label')
target_names = category_id_df.label_name.values
print(target_names)

['Negative' 'Neutral' 'Positive']


In [15]:
# Benchmark classifiers
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print("Train accuracy: ", clf.score(X_test, y_test))

    score = accuracy_score(y_test, pred)
    print("Test accuracy:   %0.3f" % score)

    # print("classification report:")
    # print(classification_report(y_test, pred, target_names=target_names ))

    # Plot confusion matrix
    # disp = ConfusionMatrixDisplay.from_estimator(
    #     clf,
    #     X_test,
    #     y_test,
    #     display_labels=target_names,
    #     cmap=plt.cm.Blues,
    #     xticks_rotation= 90
    # )
    # disp.ax_.set_title("Confusion matrix:", None)
    # print("Confusion matrix:")
    # plt.show()

    precision = precision_score(y_test, pred, average='macro')
    recall =  recall_score(y_test, pred, average='macro')
    F1_Score = f1_score(y_test, pred, average='macro')

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, precision, recall, F1_Score

In [16]:
results = []

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(penalty=penalty, dual=False,tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.000000000001, max_iter=50,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0000000000001, max_iter=50,
                                       penalty="elasticnet")))



# Train LogisticRegression classifier
print('=' * 80)
print("Logistic Regression")
results.append(benchmark(LogisticRegression(multi_class='ovr', solver='liblinear')))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.1)))
results.append(benchmark(BernoulliNB(alpha=.1)))
results.append(benchmark(ComplementNB(alpha=.01)))

print('=' * 80)
print("Naive Bayes 2")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))
results.append(benchmark(ComplementNB(alpha=.1)))

L2 penalty
________________________________________________________________________________
Training: 
LinearSVC(dual=False, tol=0.001)
train time: 1.650s
test time:  0.002s
Train accuracy:  0.7730192719486081
Test accuracy:   0.773

________________________________________________________________________________
Training: 
SGDClassifier(alpha=1e-12, max_iter=50)
train time: 0.290s
test time:  0.008s
Train accuracy:  0.7521413276231264
Test accuracy:   0.752

L1 penalty
________________________________________________________________________________
Training: 
LinearSVC(dual=False, penalty='l1', tol=0.001)
train time: 1.215s
test time:  0.002s
Train accuracy:  0.7561563169164882
Test accuracy:   0.756

________________________________________________________________________________
Training: 
SGDClassifier(alpha=1e-12, max_iter=50, penalty='l1')
train time: 0.648s
test time:  0.004s
Train accuracy:  0.7510706638115632
Test accuracy:   0.751

Elastic-Net penalty
________________________

In [17]:
print('{0:30}{1:10}{2:10}{3:10}{4:10}'.format('  algorithm', '  accuracy ', '  precision ', ' recall ', '  F1_Score'))
print('---------------------------------------------------------------------')
for r in results:
    print('{0:30}{1:10.2f}{2:10.2f}{3:10.2f}{4:10.2f}'.format(r[0],r[1], r[2], r[3], r[4]))

  algorithm                     accuracy   precision  recall     F1_Score
---------------------------------------------------------------------
LinearSVC                           0.77      0.77      0.75      0.75
SGDClassifier                       0.75      0.74      0.73      0.73
LinearSVC                           0.76      0.75      0.73      0.74
SGDClassifier                       0.75      0.74      0.72      0.73
SGDClassifier                       0.75      0.75      0.73      0.73
LogisticRegression                  0.75      0.75      0.71      0.72
MultinomialNB                       0.76      0.75      0.73      0.73
BernoulliNB                         0.73      0.73      0.71      0.71
ComplementNB                        0.74      0.73      0.72      0.72
MultinomialNB                       0.75      0.74      0.73      0.73
BernoulliNB                         0.74      0.73      0.72      0.72
ComplementNB                        0.76      0.75      0.73      0.73


In [18]:
print('{0:30}{1:10}{2:10}{3:10}{4:10}'.format('  algorithm', '  accuracy ', '  precision ', ' recall ', '  F1_Score'))
print('---------------------------------------------------------------------')
for r in results:
    print('{0:30}{1:10.3f}{2:10.3f}{3:10.3f}{4:10.3f}'.format(r[0],r[1], r[2], r[3], r[4]))

  algorithm                     accuracy   precision  recall     F1_Score
---------------------------------------------------------------------
LinearSVC                          0.773     0.768     0.748     0.753
SGDClassifier                      0.752     0.743     0.732     0.735
LinearSVC                          0.756     0.750     0.732     0.737
SGDClassifier                      0.751     0.743     0.724     0.729
SGDClassifier                      0.754     0.748     0.728     0.733
LogisticRegression                 0.747     0.751     0.710     0.716
MultinomialNB                      0.758     0.751     0.726     0.731
BernoulliNB                        0.728     0.730     0.710     0.712
ComplementNB                       0.743     0.731     0.715     0.719
MultinomialNB                      0.752     0.740     0.728     0.731
BernoulliNB                        0.738     0.727     0.722     0.722
ComplementNB                       0.758     0.748     0.730     0.734
